<a href="https://colab.research.google.com/github/parkererickson/GCNMoviePrediction/blob/master/MoviePredictionGCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="TigerGraph logo" height="45px" src="https://blobscdn.gitbook.com/v0/b/gitbook-28427.appspot.com/o/spaces%2F-LHvjxIN4__6bA0T-QmU%2Favatar.png?generation=1532158270801864&amp;alt=media" align="left" hspace="10px" vspace="0px"></p>

# Graph Convolutional Neural Networks for Movie Recommendation
------
## Introduction
This notebook walks through a basic example of using a graph convolutional neural network (GCN) for recommendation. The data is collected from a TigerGraph database using a Python package [pyTigerGraph](https://github.com/parkererickson/pyTigerGraph). Data collected is then pushed through a GCN to output predictions about a person's viewing preferences. This example does makes a couple oversimplifications that will be pointed out, mainly in the assumptions made surrounding a person's preferences. 




## Install Queries on TigerGraph Server
You need to create and install two queries on the TigerGraph server; one named userRatings and another called movieLinks.

```
CREATE QUERY userRatings(VERTEX<USER> user) FOR GRAPH Recommender { 
  /* movieID | userID | userRating | term | termRating */
  SumAccum<float> @rating;
	
	src = {user};
  
	S1 = SELECT tgt FROM src:s -(rate:e)-> MOVIE:tgt
       ACCUM tgt.@rating += e.rating;

  PRINT S1[S1.movie_id as movieID, S1.name as movieTitle, S1.@rating as userRating];
}
```

```
CREATE QUERY movieLinks() FOR GRAPH Recommender SYNTAX v2{ 
	TYPEDEF TUPLE <STRING src, STRING dest> TUPLE_RECORD;
	ListAccum<TUPLE_RECORD> @@tupleRecords;
	movies = {MOVIE.*};  
	result = SELECT tgt FROM movies:s-(:e1)-TERM:mid-(:e2)-MOVIE:tgt WHERE s != tgt 
	         ACCUM @@tupleRecords += TUPLE_RECORD (s.name, tgt.name);
	PRINT @@tupleRecords;
}
```

## Installing Packages
The core packages that need to be installed are PyTorch, dgl, and pyTigerGraph. PyTorch and dgl are used for creating and training the GCN, while pyTigerGraph is used for connecting to the TigerGraph database. We also import networkx for converting the list of edges from TigerGraph into a graph dgl can work with.

In [0]:
!pip install pyTigerGraph
!pip install torch torchvision
!pip install dgl
!pip install networkx

  Created wheel for pyTigerGraph: filename=pyTigerGraph-0.0.4.6-cp36-none-any.whl size=3410 sha256=56e6daec4f9fae2da148603883cb2a1f1e2ec086228b7c971486d34a847ac867
  Stored in directory: /root/.cache/pip/wheels/b6/3a/d3/24bb96c355fdda37c0bee211275b98a3cfc08b33432169ed26
  Created wheel for validators: filename=validators-0.14.1-cp36-none-any.whl size=17239 sha256=5b83734749e031f6ff6aab6a8a0308c8a203328f0dc58f1d0f00faae63d3a899
  Stored in directory: /root/.cache/pip/wheels/a1/fb/78/272466691f4117974f8d5811e4060fc61e1179a1c00db6f9bb
Successfully built pyTigerGraph validators
     |████████████████████████████████| 2.4MB 2.8MB/s 


## Importing Packages
We now import the packages we just installed

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pyTigerGraph as tg
import dgl
import networkx as nx
from heapq import nlargest, nsmallest


##Configuration

Here we define some variables, such as the number of epochs of training (usually only need 30 or less for a 2-layer GCN), the learning rate (0.01 seems to work well).


In [0]:
numEpochs = 25
learningRate = 0.01

## Creating the Graph Convolutional Neural Network
The block below defines some functions and classes for the GCN. The main ones to look at are the GCNLayer, which are the individual building blocks that the GCN class is made out of. The GCN class defines the structure of our neural network.

In [0]:
# Define the message and reduce function
# NOTE: We ignore the GCN's normalization constant c_ij for this tutorial.
def gcn_message(edges):
    # The argument is a batch of edges.
    # This computes a (batch of) message called 'msg' using the source node's feature 'h'.
    return {'msg' : edges.src['h']}

def gcn_reduce(nodes):
    # The argument is a batch of nodes.
    # This computes the new 'h' features by summing received 'msg' in each node's mailbox.
    return {'h' : torch.sum(nodes.mailbox['msg'], dim=1)}

# Define the GCNLayer module
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, inputs):
        # g is the graph and the inputs is the input node features
        # first set the node features
        g.ndata['h'] = inputs
        # trigger message passing on all edges
        g.send(g.edges(), gcn_message)
        # trigger aggregation at all nodes
        g.recv(g.nodes(), gcn_reduce)
        # get the result node features
        h = g.ndata.pop('h')
        # perform linear transformation
        return self.linear(h)

# Define a 2-layer GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)
        h = torch.relu(h)
        h = self.gcn2(g, h)
        return h


## Creating Database Connection and Creating Edge List

This section instantiates a connection to the TigerGraph database and creates a list of tuples which consist of directed edges in the form of (from, to). This is done through two dictionaries that corresponds an article name to a unique numerical id that is needed to process the graph in the GCN.


#### **Assumption Alert:** We oversimplify the graph here. The query returns pairs of movies that share the same term (genre). In the real world, most people like a variety of genres and therefore their views are a little more nuanced than creating a graph where the edges are created if the movies share the same genre. Better link creation factors might be actors, directors, etc. but we don't have that in this dataset. Where TigerGraph comes in is the ease of data extraction, as there are no JOIN operations to create these links between movies.
* Note: It is possible to create a GCN that has multiple types of verticies, (known as a Relational Graph Convolutional Notebook) but it is more complex. A good way to get started is to simplify until you only have relations between the same type of thing.


In [0]:
graph = tg.TigerGraphConnection(
    ipAddress="https://graphml.i.tgcloud.io", 
    graphname="Recommender", 
    apiToken="bekr9ls24mlh4kbkd7g28stq8vpj67vi") # Really not the best idea to have your API key out in the open, but for the sake of the demo, here it is

movieToNum = {} # translation dictionary for movie name to number (for dgl)
numToMovie = {} # translation dictionary for number to movie name
i = 0
def createEdgeList(result): # returns tuple of number version of edge
    global i
    if result["src"] in movieToNum:
        fromKey = movieToNum[result["src"]]
    else:
        movieToNum[result["src"]] = i
        numToMovie[i] = result["src"]
        fromKey = i
        i+=1
    if result["dest"] in movieToNum:
        toKey = movieToNum[result["dest"]]
    else:
        movieToNum[result["dest"]] = i
        numToMovie[i] = result["dest"]
        toKey = i
        i+=1
    return (fromKey, toKey)
    
edges = [createEdgeList(thing) for thing in graph.runInstalledQuery("movieLinks", {}, sizeLimit=128000000)["results"][0]["@@tupleRecords"]] # creates list of edges
print(len(edges))
print(edges[:5])

1046378
[(0, 1), (2, 1), (3, 1), (4, 1), (5, 1)]



## Initializing Graph

This section converts the list of edges into a graph that DGL can process in the GCN.

In [0]:
g = nx.Graph()
g.add_edges_from(edges)


G = dgl.DGLGraph(g)

## Adding Features to Graph
We one-hot encode the features of the verticies in the graph. Feature assignment can be done a multitude of different ways, this is just the fastest and easiest, especially given the lack of attributal information in the dataset.

If you had a graph of documents for example, you could run doc2vec on those documents to create a feature vector and create the feature matrix by concatenating those together.

Another possiblity is that you have a graph of songs, artists, albums, etc. and you could use tempo, max volume, minimum volume, length, and other numerical descriptions of the song to create the feature matrix.

In [0]:
G.ndata["feat"] = torch.eye(G.number_of_nodes())

print(G.nodes[2].data['feat'])

tensor([[0., 0., 1.,  ..., 0., 0., 0.]])


## Get User Data

In this section, we get a specific user's movie preferences. There is a lot of list comprehension going on, but just know that we are getting the user's 3 highest and lowest reviewed movies for a total of 6 labelled datapoints to feed the GCN. The remainder of the user's data is then processed and saved to test the accuracy of the GCN.

In [0]:
ratings = graph.runInstalledQuery("userRatings", {"user":"7"})["results"][0]["S1"]
print("Total Number of Reviews by User: "+str(len(ratings)))
top3Movies = [thing["attributes"]["movieTitle"] for thing in nlargest(3, ratings, key=lambda item: item["attributes"]["userRating"])] # getting the 3 highest rated movies by the user
bottom3Movies = [thing["attributes"]["movieTitle"] for thing in nsmallest(3, ratings, key=lambda item: item["attributes"]["userRating"])] # getting the 3 lowest rated movies by the user
unclassifiedMovies = [thing for thing in ratings if not((thing["attributes"]["movieTitle"] in top3Movies) or (thing["attributes"]["movieTitle"] in bottom3Movies))]

def filterNegative(thing):
  if thing["attributes"]["userRating"] < 0:
    return thing

negativeRating = [filterNegative(thing)["attributes"]["movieTitle"] for thing in unclassifiedMovies if filterNegative(thing) != None]
positiveRating = [thing["attributes"]["movieTitle"] for thing in ratings if thing["attributes"]["movieTitle"] not in negativeRating]
print("Number of movies whose rating is unknown to the GCN: "+str(len(unclassifiedMovies)))
print("Number of unknown movies with a negative rating: "+str(len(negativeRating)))
print("Number of unknown movies with a positive rating: "+str(len(positiveRating)))
print(top3Movies)
print(bottom3Movies)

Total Number of Reviews by User: 403
Number of movies whose rating is unknown to the GCN: 397
Number of unknown movies with a negative rating: 116
Number of unknown movies with a positive rating: 287
['Audrey Rose (1977)', 'NeverEnding Story III, The (1994)', 'Boxing Helena (1993)']
['Four Weddings and a Funeral (1994)', 'Unbearable Lightness of Being, The (1988)', 'Nightmare on Elm Street, A (1984)']



## Creating Neural Network and Labelling Relevant Verticies

Here, we create the GCN. A two-layered GCN appears to work better than deeper networks, and this is further corroborated by the fact [this](https://arxiv.org/abs/1609.02907) paper only used a two-layered one. We also label the wanted and unwanted verticies and setup the optimizer. Since the GCN is a semi-supervised algorithm, we do not label all of the nodes to their correct classes before training - only two are needed!


In [0]:
net = GCN(G.number_of_nodes(), 20, 2) #Two layer GCN
inputs = G.ndata["feat"]
labeled_nodes = torch.tensor([movieToNum[top3Movies[0]], movieToNum[top3Movies[1]], movieToNum[top3Movies[2]], 
                              movieToNum[bottom3Movies[0]], movieToNum[bottom3Movies[1]], movieToNum[bottom3Movies[2]]])  # only the liked movies and the disliked movies are labelled
labels = torch.tensor([0, 0, 0, 1, 1, 1])  # their labels are different
optimizer = torch.optim.Adam(net.parameters(), lr=learningRate)

## Training Loop
Below is the training loop that actually trains the GCN. Unlike many traditional deep learning architectures, GCNs don't always need that much training or as large of data sets due to their exploitation of the *structure* of the data, as opposed to only the features of the data.
* Note: due to the randomized initial values of the weights in the neural network, sometimes models don't work very well, or their loss gets stuck at a relatively large number. If that happens, just stop and restart the training process (also rerun the cell above to reset the weights) and hope for better luck!

In [0]:
all_logits = []
for epoch in range(numEpochs):
    logits = net(G, inputs)
    # we save the logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    # we only compute loss for labeled nodes
    loss = F.nll_loss(logp[labeled_nodes], labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %6.3e' % (epoch, loss.item()))

Epoch 0 | Loss: 1.356e+01
Epoch 1 | Loss: 5.604e+01
Epoch 2 | Loss: 1.797e+02
Epoch 3 | Loss: 5.909e+01
Epoch 4 | Loss: 1.083e+01
Epoch 5 | Loss: 8.827e+00
Epoch 6 | Loss: 5.314e+00
Epoch 7 | Loss: 4.826e-01
Epoch 8 | Loss: 5.298e+00
Epoch 9 | Loss: 8.436e+00
Epoch 10 | Loss: 8.925e+00
Epoch 11 | Loss: 7.530e+00
Epoch 12 | Loss: 4.470e+00
Epoch 13 | Loss: 2.316e-01
Epoch 14 | Loss: 4.095e+00
Epoch 15 | Loss: 6.054e+00
Epoch 16 | Loss: 6.115e+00
Epoch 17 | Loss: 4.500e+00
Epoch 18 | Loss: 1.413e+00
Epoch 19 | Loss: 2.964e+00
Epoch 20 | Loss: 5.250e+00
Epoch 21 | Loss: 5.706e+00
Epoch 22 | Loss: 4.552e+00
Epoch 23 | Loss: 1.979e+00
Epoch 24 | Loss: 1.850e+00


## Testing Accuracy
Here is the code that processes the GCN's results and calculates the accuracy based off the verticies that the user has reviewed, but were not labelled in the graph for the GCN to use. While this accuracy is pretty mediocre, the GCN does make predictions based off of movies sharing the same genre, and therefore with better data, there could be (and almost certainly would be) an improvement in accuracy.

In [0]:
predictions = list(all_logits[numEpochs-1])

positivePrediction = []
negativePrediction = []
a = 0
for movie in predictions:
    if movie[0] >= movie[1]:
      positivePrediction.append(numToMovie[a])
    else:
      negativePrediction.append(numToMovie[a])
    a+=1

totalPredictions = len(unclassifiedMovies)
totalRight = 0

for movie in unclassifiedMovies:
  if (movie["attributes"]["movieTitle"] in negativePrediction) and (movie["attributes"]["movieTitle"] in negativeRating):
    totalRight += 1
  if (movie["attributes"]["movieTitle"] in positivePrediction) and (movie["attributes"]["movieTitle"] in positiveRating):
    totalRight += 1
    
print("Number of movies whose rating is unknown to the GCN: "+str(len(unclassifiedMovies)))
print("Total number of correct classifications: "+str(totalRight))
print("Accuracy: "+str(totalRight/totalPredictions))

Number of movies whose rating is unknown to the GCN: 397
Total number of correct classifications: 164
Accuracy: 0.41309823677581864


## Credits
<p><img alt="Picture of Parker Erickson" height="150px" src="https://avatars1.githubusercontent.com/u/9616171?s=460&v=4" align="right" hspace="20px" vspace="20px"></p>

Demo/tutorial written by Parker Erickson, a student at the University of Minnesota pursuing a B.S. in Computer Science. His interests include graph databases, machine learning, travelling, playing the saxophone, and watching Minnesota Twins baseball. Feel free to reach out! Find him on:

* LinkedIn: [https://www.linkedin.com/in/parker-erickson/](https://www.linkedin.com/in/parker-erickson/)
* GitHub: [https://github.com/parkererickson](https://github.com/parkererickson)
* Medium: [https://medium.com/@parker.erickson](https://medium.com/@parker.erickson)
* Email: parker.erickson30@gmail.com
----
GCN Resources:
* DGL Documentation: [https://docs.dgl.ai/](https://docs.dgl.ai/)
* GCN paper by Kipf and Welling [https://arxiv.org/abs/1609.02907](https://arxiv.org/abs/1609.02907)
* R-GCN paper: [https://arxiv.org/abs/1703.06103](https://arxiv.org/abs/1703.06103)
---- 
Notebook adapted from: [https://docs.dgl.ai/en/latest/tutorials/basics/1_first.html](https://docs.dgl.ai/en/latest/tutorials/basics/1_first.html)